In [23]:
from datasets import load_dataset
from transformers import pipeline
import torch

In [24]:
dataset = load_dataset('davidgaofc/PRIMA_inout')


In [25]:
def truncate_sequences(batch):
    # Assume 'text' is the key in your dataset that contains the text you want to truncate
    # You need to adjust this if your dataset uses a different key
    truncated_text = [text[:512] for text in batch['Text']]
    return {'Truncated_text': truncated_text}

trunc_dataset = dataset.map(truncate_sequences, batched = True)

In [26]:
model_name = "davidgaofc/hh-labeler"
classify = pipeline("text-classification", model=model_name)

In [27]:
def classify_and_rearrange(entry):
    result = classify(entry['Truncated_text'])[0]
    human_part, assistant_part = entry['Text'].split('assistant:')

    parts = assistant_part.split("2. ")
    part1 = parts[0].split("1.")[1]
    part2 = parts[1]

    if result['label'] == "LABEL_1":
        column1 = human_part + ' assistant: ' + part2
        column2 = human_part + ' assistant: ' + part1
    else:
        column1 = human_part + ' assistant: ' + part1
        column2 = human_part + ' assistant: ' + part2

    return {'chosen': column1, 'rejected': column2}

In [28]:
processed_dataset = trunc_dataset.map(classify_and_rearrange)

Map:   0%|          | 0/1640 [00:00<?, ? examples/s]

In [29]:
processed_dataset = processed_dataset.remove_columns(['Text'])
processed_dataset = processed_dataset.remove_columns(['Label'])
processed_dataset = processed_dataset.remove_columns(['Truncated_text'])


In [30]:
processed_dataset.push_to_hub("davidgaofc/PRIMA_RM_train_format")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

In [31]:
processed_dataset

DatasetDict({
    train: Dataset({
        features: ['chosen', 'rejected'],
        num_rows: 1640
    })
})

In [32]:
processed_dataset['train'][0]

{'chosen': 'human: what are the subject of gastroschisis?  what are the signs and symptoms of gastroschisis? the human phenotype infection ( hpo ) is a rare condition that can cause a lot of injuries and symptoms such as a loss of appetite, fatigue\n assistant: what are the subject of gastroschisis? what are the signs and symptoms of gastroschisis? the human phenotype infection ( hpo ) is a rare condition that can cause a lot of injuries and symptoms such as a loss of appetite, fatigue s are the cause. symptoms of which the human phenotype infection. is a rare condition that can cause a lot of, and symptoms such as a result of appetite, and',
 'rejected': 'human: what are the subject of gastroschisis?  what are the signs and symptoms of gastroschisis? the human phenotype infection ( hpo ) is a rare condition that can cause a lot of injuries and symptoms such as a loss of appetite, fatigue\n assistant:  what are the subject of gastroschisis? what are the signs and symptoms of gastroschi